In [ ]:
!pip install --upgrade PyGithub tqdm arrow python-dotenv

In [1]:
from github import Github, RateLimitExceededException
import pandas as pd
import json
from tqdm.notebook import trange, tqdm
import arrow
from time import sleep
import pickle
from dotenv import load_dotenv
import os

load_dotenv()
GITHUB_KEY = os.getenv("GITHUB_KEY")

In [2]:
g = Github(GITHUB_KEY)

In [3]:
#arrow.get(g.rate_limiting_resettime)

In [4]:
g.rate_limiting

(4996, 5000)

In [5]:
repo = g.get_repo("kubernetes/kubernetes")

# Releases

In [6]:
github_releases = repo.get_releases()
total_releases = github_releases.totalCount
print(total_releases)

353


In [10]:
try:
    with open('../resources/releases.pickle', 'rb') as handle:
        releases = pickle.load(handle)
        print(f"Loaded {len(releases.values())} commits from pickle")
except:
    print("Fail in load releases pickle, commit will be empty")
    releases = {}

Fail in load releases pickle, commit will be empty


In [25]:
for i in trange(int(total_releases/30)+1, desc='GitHub Releases'):
    #if i not in github_releases.get_page(i):
        #commit_pages[i] = github_commits.get_page(i)
    for c_releases in github_releases.get_page(i):
        tag_name = c_releases.tag_name
        if tag_name not in releases:
            releases[tag_name] = c_releases.raw_data
            sleep(1)
        else:
            break

In [26]:
print(f"Now there are {len(releases.values())} commits on pickle")

Now there are 353 commits on pickle


In [28]:
with open('../resources/releases.pickle', 'wb') as handle:
    pickle.dump(releases, handle, protocol=pickle.HIGHEST_PROTOCOL)